# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
nltk.download(['punkt','stopwords', 'wordnet'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dengl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dengl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dengl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterRespond.db')
df = pd.read_sql('cleaned_data',con = engine)


In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message'].values
y = df[df.columns[4:]]

In [5]:
def load_data():
    '''
    Load data from sqlite database and set X and y for modeling preparation
    
    OUTPUT
        - X: array of messages
        - y: target dataframe
    '''
    engine = create_engine('sqlite:///DisasterRespond.db')
    df = pd.read_sql('cleaned_data',con = engine)
    X = df['message'].values
    y = df[df.columns[4:]]
    return X, y

In [6]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [8]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [9]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [10]:
def tokenize(text):
    '''
        Processed text in string format, and output a list of words
        INPUT:
            -text: string, raw text data
        OUTPUT:
            -clean_tokens, list of processed words
    
    '''
    # get list of all urls using regex
    detected_urls = re.findall(url_regex,text)
    # replace each url in text strings with placeholder
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    
    # Case Normalization
    text = text.lower() # convert to lowercase
    # remove puntuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ",text)
    
    # tokenize text
    tokens = word_tokenize(text)
    token_list =[]
    # remove stop words
    for tok in tokens:
        if tok not in stopwords.words("english"):
             token_list.append(tok)
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iritate through each token
    clean_tokens = []
    for tok in token_list:
        # lemmatize and remove leading and tailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
        
        clean_tokens.append(clean_tok)
    return clean_tokens


In [11]:
print(tokenize(X[3]))

['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [20]:
# create the pipeline
parameters = 
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf',MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
]) # n_jobs=-1 means using all available processes / thread

In [21]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)),
 'clf__estimator': RandomForestClassifier(n_jobs=-1),
 'clf__estimator__bootstrap': True,
 'clf__estimator__ccp_alpha': 0.0,
 'clf__estimator__class_weight': None,
 'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': None,
 'clf__estimator__max_features': 'auto',
 'clf__estimator__max_leaf_nodes': None,
 'clf__estimator__max_samples': None,
 'clf__estimator__min_impurity_decrease': 0.0,
 'clf__estimator__min_impurity_split': None,
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__min_samples_split': 2,
 'clf__estimator__min_weight_fraction_leaf': 0.0,
 'clf__estimator__n_estimators': 100,
 'clf__estimator__n_jobs': -1,
 'clf__estimator__oob_score': False,
 'clf__estimator__random_state': None,
 'clf__estimator__verbose': 0,
 'clf__estimator__warm_start': False,
 'clf__n_jobs': None,
 'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001BF111E3D90>

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [24]:
# train test data split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=42)

# train classifier
pipeline.fit(X_train,y_train)
# predict test data
y_pred = pipeline.predict(X_test)

In [25]:
# calculate accuracy
accuracy = (y_pred == y_test).mean()

print("Accuracy:", accuracy)

Accuracy: related                   0.812023
request                   0.888923
offer                     0.984590
aid_related               0.763656
medical_help              0.912878
medical_products          0.941410
search_and_rescue         0.967043
security                  0.971926
military                  0.962313
child_alone               0.990998
water                     0.947818
food                      0.926915
shelter                   0.930882
clothing                  0.977724
money                     0.972078
missing_people            0.980928
refugees                  0.958193
death                     0.953311
other_aid                 0.859323
infrastructure_related    0.926457
transport                 0.947360
buildings                 0.944919
electricity               0.969027
tools                     0.982911
hospitals                 0.980928
shops                     0.986421
aid_centers               0.978944
other_infrastructure      0.947360
weather_re

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [25]:
def get_scores(y_test,y_pred):
    i =0
    for col in y_test:
        print('Feature {}:{}'.format(i+1,col))
        print(classification_report(y_test[col],y_pred[:,i]))
        i=i+1
    accuracy = (y_pred == y_test.values).means()
    print('The model accuracy score is {:.3f}'.format(accuracy))

### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
parameters = parameters = {
    'clf__estimator__max_depth': [15, 25],
    'clf__estimator__n_estimators': [100,200]
        }
# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters)

In [28]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=1, target_names=y.columns))  # target_names=target_names

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4991
               request       1.00      0.00      0.00      1158
                 offer       1.00      0.00      0.00        80
           aid_related       0.91      0.21      0.34      2717
          medical_help       1.00      0.00      0.00       582
      medical_products       1.00      0.00      0.00       391
     search_and_rescue       1.00      0.00      0.00       206
              security       1.00      0.00      0.00       163
              military       1.00      0.00      0.00       247
           child_alone       1.00      0.00      0.00        47
                 water       1.00      0.00      0.00       465
                  food       1.00      0.00      0.00       792
               shelter       0.00      0.00      0.00       628
              clothing       1.00      0.00      0.00       145
                 money       1.00      

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [32]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin

In [27]:
from sklearn.pipeline import FeatureUnion

In [28]:
from nltk.stem import PorterStemmer

In [29]:
class DisasterWordExtractor(BaseEstimator, TransformerMixin):

    def disaster_words(self, text):
        """
        INPUT: text - string, raw text data
        OUTPUT: bool -bool object, True or False
        """
        # Build a list of words that are constantly used during a disaster event
        words = ['food','hunger','hungry','starving','water','drink','eat','thrist',
            'need','hospital','medicine','medical','ill','pain','disease','injured','falling',
            'wound','dying','death','dead','aid','help','assistance','cloth','cold','wet','shelter',
                'hurricane','earthquake','flood','live','alive','child','people','shortage','blocked',
                 'gas','pregnant','baby'
        ]
        
        
        # lemmatize the words
        lemmatized_words = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
        # Get the stem words of each word in  lemmatized_words 
        stem_disaster_words = [PorterStemmer().stem(w) for w in lemmatized_words]
       
        # get list of all urls using regex
        detected_urls = re.findall(url_regex,text)
        # replace each url in text strings with placeholder
        for url in detected_urls:
            text = text.replace(url, 'urlplaceholder')
            
        #tokenize the text
        clean_tokens = tokenize(text)
        for token in clean_tokens:
            if token in stem_disaster_words:
                return True
        return False
        
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_disaster_word = pd.Series(X).apply(self.disaster_words)
        return pd.DataFrame(X_disaster_word)



In [92]:
# create the pipeline

dis_pipeline = Pipeline([
        ('features',FeatureUnion([
        ('text_pipeline',Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer())])),
        ('disaster_words',DisasterWordExtractor())
        ])),
        ('clf',MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
        ])
        
        

In [96]:
# train classifier
dis_pipeline.fit(X_train,y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000001BF111E3D90>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('disaster_words',
                                                 DisasterWordExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

In [97]:
dis_y_pred = dis_pipeline.predict(X_test)

In [98]:
print(classification_report(y_test, y_pred, zero_division=1, target_names=y.columns))  # target_names=target_names

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4991
               request       1.00      0.00      0.00      1158
                 offer       1.00      0.00      0.00        80
           aid_related       0.91      0.21      0.34      2717
          medical_help       1.00      0.00      0.00       582
      medical_products       1.00      0.00      0.00       391
     search_and_rescue       1.00      0.00      0.00       206
              security       1.00      0.00      0.00       163
              military       1.00      0.00      0.00       247
           child_alone       1.00      0.00      0.00        47
                 water       1.00      0.00      0.00       465
                  food       1.00      0.00      0.00       792
               shelter       0.00      0.00      0.00       628
              clothing       1.00      0.00      0.00       145
                 money       1.00      

In [99]:
# create k neiboughood pipeline
k_nhb_pipeline = Pipeline([
        ('features',FeatureUnion([
        ('text_pipeline',Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer())])),
        ('disaster_words',DisasterWordExtractor())
        ])),
        ('clf',MultiOutputClassifier(estimator = KNeighborsClassifier(n_jobs=-1)))
        ])

In [100]:
# train classifier

k_nhb_pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000001BF111E3D90>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('disaster_words',
                                                 DisasterWordExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_jobs=-1)))])

In [102]:
y_pred_k_khb = k_nhb_pipeline.predict(X_test)

In [104]:
category_names = y.columns.tolist()

In [129]:
for idx in range(len(category_names)):
    print('Category: {} '.format(category_names[idx]))
    print(classification_report(y_test.iloc[:,idx].values, y_pred_k_khb[:,idx], zero_division='warn'))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, idx].values, y_pred_k_khb[:, idx])))
    print("------------------------------------------------------\n")

Category: related 
              precision    recall  f1-score   support

           0       0.63      0.14      0.23      1563
           1       0.78      0.97      0.87      4991

    accuracy                           0.78      6554
   macro avg       0.71      0.56      0.55      6554
weighted avg       0.75      0.78      0.72      6554

Accuracy 0.7754043332316143


------------------------------------------------------

Category: request 
              precision    recall  f1-score   support

           0       0.92      0.86      0.89      5396
           1       0.50      0.65      0.56      1158

    accuracy                           0.82      6554
   macro avg       0.71      0.75      0.73      6554
weighted avg       0.84      0.82      0.83      6554

Accuracy 0.8225511138236191


------------------------------------------------------

Category: offer 
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6474
          

              precision    recall  f1-score   support

           0       0.84      0.79      0.82      4734
           1       0.54      0.62      0.57      1820

    accuracy                           0.75      6554
   macro avg       0.69      0.71      0.70      6554
weighted avg       0.76      0.75      0.75      6554

Accuracy 0.7450411962160512


------------------------------------------------------

Category: floods 
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      5988
           1       0.40      0.37      0.38       566

    accuracy                           0.90      6554
   macro avg       0.67      0.66      0.66      6554
weighted avg       0.89      0.90      0.90      6554

Accuracy 0.8973146170277693


------------------------------------------------------

Category: storm 
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      5902
           1       0.46      0

In [139]:
parameters = {'clf__estimator__n_estimators': [100, 150]}
# create grid search object
knhb_cv = GridSearchCV(k_nhb_pipeline, param_grid=parameters, scoring='recall_micro')

In [125]:
from sklearn.metrics import accuracy_score


In [111]:
def get_scores(y_true, y_pred):
    """
    Returns the accuracy, precision and recall and f1 scores of the two same shape numpy arrays `y_true` and `y_pred`.

    INPUTS:
        y_true - Numpy array object - A (1 x n) vector of true values
        y_pred - Numpy array object - A (1 x n) vector of predicted values
        
    OUPUT:
        dict_scores - Python dict - A dictionary of accuracy, precision and recall and f1 scores of `y_true` and `y_pred`.
    """
    
    # Compute the accuracy score of y_true and y_pred
    accuracy = accuracy_score(y_true, y_pred)
    
    # Compute the precision score of y_true and y_pred
    precision =round( precision_score(y_true, y_pred, average='micro'))
    
    # Compute the recall score of y_true and y_pred
    recall = recall_score(y_true, y_pred, average='micro')
    
    # Compute the recall score of y_true and y_pred
    f_1 = f1_score(y_true, y_pred, average='micro')
    
    # A dictionary of accuracy, precision and recall and f1 scores of `y_true` and `y_pred`
    dict_scores = {
        'Accuracy': accuracy, 
        'Precision': precision, 
        'Recall': recall, 
        'F1 Score': f_1
    }
    
    return dict_scores

In [116]:
tabulate_metric_scores= lambda y_test, y_pred : pd.DataFrame([get_scores(y_test.iloc[:, idx].values, y_pred[:, idx]) for idx in range(len(category_names))], index=category_names)

In [118]:
from xgboost import XGBClassifier

In [119]:
xgb_pipeline = Pipeline([
        ('features',FeatureUnion([
        ('text_pipeline',Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer())])),
        ('disaster_words',DisasterWordExtractor())
        ])),
        ('clf',MultiOutputClassifier(estimator = XGBClassifier(max_depth=6)))
        ])

In [120]:
xgb_pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000001BF111E3D90>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('disaster_words',
                                                 DisasterWordExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(max_depth=6)))])

In [121]:
y_pred_xgb = xgb_pipeline.predict(X_test)

In [127]:
for idx in range(len(category_names)):
    print('Category: {} '.format(category_names[idx]))
    print(classification_report(y_test.iloc[:,idx].values, y_pred_xgb[:,idx], zero_division='warn'))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, idx].values, y_pred_xgb[:, idx])))
    print("------------------------------------------------------\n")

Category: related 
              precision    recall  f1-score   support

           0       0.71      0.25      0.37      1563
           1       0.81      0.97      0.88      4991

    accuracy                           0.80      6554
   macro avg       0.76      0.61      0.63      6554
weighted avg       0.78      0.80      0.76      6554

Accuracy 0.797528227036924


------------------------------------------------------

Category: request 
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5396
           1       0.83      0.48      0.61      1158

    accuracy                           0.89      6554
   macro avg       0.87      0.73      0.77      6554
weighted avg       0.89      0.89      0.88      6554

Accuracy 0.8915166310649985


------------------------------------------------------

Category: offer 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6474
           

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6441
           1       0.61      0.17      0.26       113

    accuracy                           0.98      6554
   macro avg       0.80      0.58      0.63      6554
weighted avg       0.98      0.98      0.98      6554

Accuracy 0.9838266707354287


------------------------------------------------------

Category: earthquake 
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5917
           1       0.88      0.75      0.81       637

    accuracy                           0.97      6554
   macro avg       0.93      0.87      0.90      6554
weighted avg       0.96      0.97      0.96      6554

Accuracy 0.9658223985352457


------------------------------------------------------

Category: cold 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6366
           1       0.76    

In [132]:
parameters = {'clf__estimator__n_estimators': [100, 150]}
# create grid search object
xgb_cv = GridSearchCV(xgb_pipeline, param_grid=parameters, scoring='recall_micro', cv=4)

In [133]:
xgb_cv.fit(X_train, y_train)


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x000001BF111E3D90>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('disaster_words',
                                                                        DisasterWordExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=XGBClassifier(max_depth=6)))]),
             param_grid={'clf_

In [134]:
y_pred = xgb_cv.predict(X_test)

### 9. Export your model as a pickle file

In [135]:
xgb_cv.best_score_

0.560840944696552

In [136]:
xgb_cv.best_params_

{'clf__estimator__n_estimators': 150}

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.